# import libraies

In [ ]:
# load libaries and mount to google drive
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Data_Folder_All = '/content/drive/.shortcut-targets-by-id/1g1gPWCguvifS_iicb0DszOqdo702w2Kz/productDownload_2022-10-24T192147'

# load data

In [ ]:
def get_file_path(year, table_name):

  for file_name in os.listdir(Data_Folder_All):
    if str(year) in file_name and 'csv' in file_name and 'Data' in file_name and table_name in file_name:
      return os.path.join(Data_Folder_All,file_name)

In [ ]:
def rename_cols(df, year, table_name):
  col_name_change = {}
  for name in df.columns.values:
    col_name_change[name] = '{}!!{}!!'.format(year,table_name) + name
  return df.rename(columns=col_name_change)

In [ ]:
# read all csv from list of file_names and concat together
def get_df(year_list: list, table_list: list, skiprows=[0], index_col='Geographic Area Name', axis=1):
  df_list = []

  for year in year_list:
    for table in table_list:
      df = pd.read_csv(get_file_path(year, table), skiprows=skiprows, index_col=index_col)
      df = rename_cols(df, year, table)
      df_list.append(df)

  return pd.concat(df_list, axis=axis)

# data cleanups

In [ ]:
def DP_df_cleanup(df):

  # remove Annotation and Unnamed cols
  df_cleaned1 = df.loc[:,~df.columns.str.contains("Annotation|Unnamed|Geography")]

  # remove rows with NANs
  df_cleaned2 = df_cleaned1[1 - np.sum(df_cleaned1.isnull(), 1) > 0]

  # only keep the Percent cols
  df_cleaned3 = df_cleaned2.loc[:,df_cleaned2.columns.str.contains("Percent")]

  # remove all the cols with values == (X)
  df_cleaned4 = df_cleaned3.loc[:,~(np.sum(df_cleaned3 == '(X)',0) > 0)]

  # remove all the cols with values == N
  df_cleaned5 = df_cleaned4.loc[:,~(np.sum(df_cleaned4 == 'N',0) > 0)]

  # remove all the cols with values == -
  df_cleaned6 = df_cleaned5.loc[:,~(np.sum(df_cleaned5 == '-',0) > 0)]

  # only keep the Margin of Error cols
  df_cleaned7 = df_cleaned6.loc[:,~df_cleaned6.columns.str.contains('Percent Margin of Error')]

  # set the data type to float
  df_cleaned_final = df_cleaned7.astype(float)

  return df_cleaned_final

# build data extraction tool

In [ ]:
def get_dict(df):
  cols_dict = {}

  for col_name in df.columns:
    cat_list = col_name.split('!!')
    current_dic = cols_dict

    for i, cat in enumerate(cat_list):
      if i == (len(cat_list)-1):
        if cat in current_dic:
          current_dic['col_name'] = col_name
        else:
          current_dic[cat] = {}
          current_dic = current_dic[cat]
          current_dic['col_name'] = col_name
        continue

      if cat in current_dic:
        current_dic = current_dic[cat]
      else:
        current_dic[cat] = {}
        current_dic = current_dic[cat]

  return cols_dict


In [ ]:
def direct_into(cat_list: list):
  starting_dict = COLS_DICT

  for cat in cat_list:
    starting_dict = starting_dict[cat]

  return starting_dict

In [ ]:
def show_options(cat_list: list):
  return direct_into(cat_list).keys()

In [ ]:
def get_col_names_rec(dic, rtn_list:list):
  for k in dic.keys():
    if k == 'col_name':
      rtn_list.append(dic[k])
    else:
      get_col_names_rec(dic[k], rtn_list)

In [ ]:
def get_col_names(cat_list: list):
  rtn_list = []
  starting_dict = direct_into(cat_list)
  get_col_names_rec(starting_dict, rtn_list)
  return rtn_list

In [ ]:
def get_df_by_cat(cat_list: list):
  return df.loc[:, get_col_names(cat_list)]

# working

In [ ]:
TABLE_NAMES = ['DP02', 'DP03', 'DP04', 'DP05']

In [ ]:
df = get_df([2011, 2021])
df = DP_df_cleanup(df)

In [ ]:
COLS_DICT = get_dict(df)

In [ ]:
show_options(['2011', 'DP02', 'Percent'])

dict_keys(['HOUSEHOLDS BY TYPE', 'RELATIONSHIP', 'MARITAL STATUS', 'SCHOOL ENROLLMENT', 'EDUCATIONAL ATTAINMENT', 'VETERAN STATUS', 'DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION', 'RESIDENCE 1 YEAR AGO', 'PLACE OF BIRTH'])

In [ ]:
show_options(['2011', 'DP03', 'Percent'])

dict_keys(['EMPLOYMENT STATUS', 'CLASS OF WORKER', 'INCOME AND BENEFITS (IN 2011 INFLATION-ADJUSTED DOLLARS)', 'HEALTH INSURANCE COVERAGE', 'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL'])

In [ ]:
for table_name in

# scrap

In [ ]:
unemployment_rate_2021 = get_df_by_cat(['2021', 'DP03', 'Percent', 'EMPLOYMENT STATUS', 'Civilian labor force', 'Unemployment Rate']).iloc[:,0]

In [ ]:
poverty_rate_2021 = get_df_by_cat(['2021', 'DP03', 'Percent', 'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL', 'All people']).iloc[:,0]

In [ ]:
unemployment_rate_2011 = get_df_by_cat(['2011', 'DP03', 'Percent', 'EMPLOYMENT STATUS', 'Percent Unemployed']).iloc[:,0]

In [ ]:
poverty_rate_2011 = get_df_by_cat(['2011', 'DP03', 'Percent', 'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL', 'All people']).iloc[:,0]

In [ ]:
population_2011 = get_df_by_cat(['2011', 'DP02', 'Percent', 'PLACE OF BIRTH','Total population']).iloc[:,0]

In [ ]:
population_2021 = get_df_by_cat(['2021', 'DP02', 'Percent', 'PLACE OF BIRTH','Total population']).iloc[:,0]

In [ ]:
unemployment_rate_change = unemployment_rate_2021 - unemployment_rate_2011

In [ ]:
poverty_rate_change = poverty_rate_2021 - poverty_rate_2011

In [ ]:
population_change = population_2021 - population_2011

In [ ]:
plt.scatter(population_2011, unemployment_rate_change.abs())

In [ ]:
plt.scatter(np.log10(population_2011), unemployment_rate_change)

In [ ]:
plt.scatter(np.log10(population_2011), poverty_rate_change)

In [ ]:
plt.scatter(unemployment_rate_change, poverty_rate_change)

In [ ]:
plt.hist(unemployment_rate_change)

In [ ]:
plt.scatter(unemployment_rate_2011, poverty_rate_2011)

In [ ]:
plt.scatter(unemployment_rate_2021, poverty_rate_2021)

In [ ]:
plt.scatter(unemployment_rate_2021, unemployment_rate_2011)

In [ ]:
plt.scatter(poverty_rate_2011, poverty_rate_2021)

In [ ]:
plt.scatter(np.log10(population_change), unemployment_rate_change)

# remove heavily correlated features

In [1]:
# Define a function
def heavily_correlated_features(df, threshold=0.8):
  """
  For each feature in "df", this function counts the number of features that
  have a correlation coefficient with that is higher than "threshold".

  Parameters
  ----------
    df: pandas DataFrame type
        Contains the features for several data points
    threshold: float type
        he threshold for which "heavily correlated" is defined.
  """

  corr = df.corr().abs()  # calculate the correlation matrix
  corr = corr[corr > threshold]  # a mask of features that are heavily correlated

  # Print out the "heavily correlated" counts
  return corr.count().sort_values(ascending=False) - 1

In [2]:
corr_count = heavily_correlated_features(df)

NameError: name 'df' is not defined

In [ ]:
COLUMNS_TO_REMOVE = corr_count[corr_count > 10].index.values

In [ ]:
df = df.drop(columns=COLUMNS_TO_REMOVE)

# EDA

## prepare df

In [ ]:
df = df_2021_MSA_data_cleaned_final.copy()

In [ ]:
df['Geographic Area Name'] = df_2021_MSA_data['Geographic Area Name'].iloc[:,0]

## total population included in out dataset (Approximatly)

In [ ]:
df['Percent!!RACE!!Total population'].sum()

that's almost the entire population of us

## differences between micro and metro areas

In [ ]:
# get a specific col for metro/mecro
def get_col_for(metro_or_mecro, col_name):
  return df.loc[df['Geographic Area Name'].str.contains(metro_or_mecro), col_name]


In [ ]:
metro_population = get_col_for('Metro', 'Percent!!RACE!!Total population')
micro_population = get_col_for('Micro', 'Percent!!RACE!!Total population')

In [ ]:
hist, bins, _ = plt.hist(metro_population, bins=10)

In [ ]:
logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))

plt.hist(metro_population, bins=logbins)
plt.xscale('log')
plt.show()

In [ ]:
plt.hist(micro_population)

# data normalization

In [ ]:
# normalize each cols
scaler = StandardScaler()
df_norm = df.copy()
df_norm[:] = scaler.fit_transform(df)

# some simple model

In [ ]:
# this X only contains the data in table DP02
X = df.drop(columns = ['Percent_change'])

In [ ]:
# Unemployment Rate col
y = df.loc[:,df.columns.str.contains('Unemployment Rate')]

In [ ]:
y = df_norm.loc[:, 'Percent_change']

In [ ]:
lasso = LassoCV(max_iter=9999999)
lasso.fit(X, y.values.ravel())

# calculate the score
score = lasso.score(X, y)
print(f'The score is {score:.3}')

# calculate betas
betas = pd.Series(lasso.coef_, index=X.columns)
betas = betas.append(pd.Series({"Intercept": lasso.intercept_}))
print(betas)

ValueError: ignored

In [ ]:
betas.sort_values()

Percent!!RELATIONSHIP!!Householder                                                                     -0.103834
Percent!!RESIDENCE 1 YEAR AGO!!Different house in the U.S.!!Different county!!Different state          -0.064535
Percent!!HOUSEHOLDS BY TYPE!!Family households (families)!!Male householder, no wife present, family   -0.044463
Percent!!EDUCATIONAL ATTAINMENT!!Graduate or professional degree                                       -0.044280
Percent!!HOUSEHOLDS BY TYPE!!Nonfamily households!!Householder living alone!!65 years and over         -0.032686
                                                                                                          ...   
Percent!!MARITAL STATUS!!Now married, except separated                                                  0.034095
Percent!!SCHOOL ENROLLMENT!!Kindergarten                                                                0.063540
Percent!!RESIDENCE 1 YEAR AGO!!Different house in the U.S.                                      

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y, lasso.predict(X))

0.3574225531688523

In [ ]:
y

Geographic Area Name
Aberdeen, WA Micro Area                        -0.110220
Abilene, TX Metro Area                          0.038848
Adrian, MI Micro Area                          -0.820242
Akron, OH Metro Area                           -0.581977
Albany, GA Metro Area                          -1.358542
                                                  ...   
York-Hanover, PA Metro Area                    -0.091110
Youngstown-Warren-Boardman, OH-PA Metro Area   -1.466100
Yuba City, CA Metro Area                        0.201039
Yuma, AZ Metro Area                             0.748837
Zanesville, OH Micro Area                      -0.562342
Name: Percent_change, Length: 390, dtype: float64